In [10]:
from models import ModelManager
modelm = ModelManager()
modelm.load_model_by_id('e_coli_core')
model = modelm.get_current_model()

#### Set Objective for Model

In [2]:
from models import ModelManager
modelm = ModelManager()
modelm.load_model_by_id('MODEL1507180061')
model = modelm.get_current_model()
print(model.reactions[:10])

[<Reaction MATP at 0x19365172bf0>, <Reaction TRANS1 at 0x19365172d40>, <Reaction TRANS2 at 0x19365173010>, <Reaction TRANS3 at 0x19365173190>, <Reaction TRANS4 at 0x19365173340>, <Reaction TRANS5 at 0x193651734f0>, <Reaction TRANS6 at 0x193651736a0>, <Reaction TRANS7 at 0x19365173850>, <Reaction TRANS8 at 0x19365173a00>, <Reaction TRANS9 at 0x19365173bb0>]


Testing


In [2]:
from cobra.io import read_sbml_model
model = read_sbml_model(r"E:\INTERNSHIP\IITM\metabolic\uploads\e_coli_core.xml")

In [3]:
print(model.objective)
print(model.objective.expression)

Maximize
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5


In [4]:
model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM")

Reaction identifier,BIOMASS_Ecoli_core_w_GAM
Name,Biomass Objective Function with GAM
Memory address,0x1b904e7da20
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP C10H12N5O13P3 + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [32]:
for rxn in model.reactions:
    if "BIOMASS" in rxn.id:
        print(rxn.id)

BIOMASS_Ecoli_core_w_GAM


In [11]:
# SET OBJECTIVE
model.objective = model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM") # RXN Object
model.objective = "BIOMASS_Ecoli_core_w_GAM" # RXN String

# SET CUSTOM OBJECTIVE
objective_dict = {
    model.reactions.get_by_id("ATPM"): 1.0,
    model.reactions.get_by_id("EX_o2_e"): -0.5,
}
model.objective = objective_dict

print(objective_dict)

# SET DIRECTION
model.objective.direction = "max"
model.objective.direction = "min"

{<Reaction ATPM at 0x1b904e4b400>: 1.0, <Reaction EX_o2_e at 0x1b904ec0bb0>: -0.5}


In [15]:
from cobra.util.solver import linear_reaction_coefficients
coeffs = linear_reaction_coefficients(model)
coeffs

{<Reaction ATPM at 0x20beddd2cb0>: 1.0,
 <Reaction EX_o2_e at 0x20bede481c0>: -0.5}

Task Tool

In [ ]:
model.objective = {
    model.reactions.get_by_id("ATPM"): 1.0,
    model.reactions.get_by_id("EX_o2_e"): -0.5,
}
model.objective.direction = "max"
print(model.objective)

Maximize
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5


In [ ]:
# HAVE A DICTIONARY FOR STORING THE { RXN ID: COEFF } IN THE MODEL MANAGER.
# UPON USER'S REQUEST KEEP ADDING BASED ON THIS.
# DIRECTION SETTING MUST ALSO BE TAKEN CARE OF.
# THEN CALL SOLUTION

# OR HAVE THE USERS ENTER DATA INTO THE UI OF THE APP AND MODEL IT.

In [ ]:
def set_model_objective(objective_dict: dict[str: int], direction="max") -> dict[str:str]:
    try:
        model = model_manager.get_current_model()
        model.objective.set_linear_coefficients(objective_dict)
        model.objective.direction = direction

        # Return confirmation
        return {
            "status": "Objective set successfully.",
            "objective": model.objective.expression,
            "direction": model.objective.direction
        }

    except Exception as e:
        return {"error": str(e)}



FVA Simulation

In [3]:
from cobra.flux_analysis import flux_variability_analysis
# type(flux_variability_analysis(model, model.reactions[:10]))
for i in range(6):
    print(model.reactions[i].name)

Phosphofructokinase
Pyruvate formate lyase
Glucose-6-phosphate isomerase
Phosphoglycerate kinase
6-phosphogluconolactonase
Acetaldehyde dehydrogenase (acetylating)


In [8]:
def flux_variability_analysis_method(rxn_names, fraction_of_optimum=0.9):
    """
    Runs Flux Variability Analysis (FVA) using user-provided reaction names.
    Matches names to reactions dynamically without building a full mapping.
    Returns a DataFrame with columns: Reaction Name, Reaction ID, Maximum Flux, Minimum Flux.
    """
    try:
        # model = model_manager.get_current_model()
        rxn_obj_list = []

        for name in rxn_names:
            match = next(
                (rxn for rxn in model.reactions if rxn.name.lower() == name.lower()),
                None
            )
            if match is None:
                raise ValueError(f"Reaction name '{name}' not found in model.")
            rxn_obj_list.append(match)

        fva_result = flux_variability_analysis(model, rxn_obj_list, fraction_of_optimum=fraction_of_optimum)

        # Convert to DataFrame and include Reaction Name as the first column
        fva_df = fva_result.reset_index()
        fva_df.insert(0, "Reaction Name", [rxn.name for rxn in rxn_obj_list])
        fva_df.columns = ["Reaction Name", "Reaction ID", "Maximum Flux", "Minimum Flux"]

        return {
            "fraction_of_optimum": fraction_of_optimum,
            "fva_output": fva_df.to_dict()
        }

    except Exception as e:
        return {"error": str(e)}
    
flux_variability_analysis_method(['Phosphofructokinase', 'Pyruvate formate lyase', 'Glucose-6-phosphate isomerase'], 0.8)

{'fraction_of_optimum': 0.8,
 'fva_output': {'Reaction Name': {0: 'Phosphofructokinase',
   1: 'Pyruvate formate lyase',
   2: 'Glucose-6-phosphate isomerase'},
  'Reaction ID': {0: 'PFK', 1: 'PFL', 2: 'PGI'},
  'Maximum Flux': {0: 0.0, 1: 0.0, 2: -18.265812670359576},
  'Minimum Flux': {0: 43.10390556972818,
   1: 19.059968898518168,
   2: 9.85667687285718}}}

In [ ]:
# def flux_variability_analysis_method(rxn_list, fraction_of_optimum=0.9):
#     """
#     Runs Flux Variability Analysis (FVA) on the model given a Reaction List and a Fraction of Optimum (FO) Value.
#     """
#     try:
#         model = model_manager.get_current_model()
#         rxn_obj_list = [model.reactions.get_by_id(i) for i in rxn_list]
#         fva_dict = flux_variability_analysis(model, rxn_obj_list).to_dict()

#         return {
#             "fraction_of_optimum": fraction_of_optimum,
#             "fva_output": fva_dict,
#         }

#     except Exception as e:
#         return {"error" : str(e)}

In [7]:
import os
def run_fva(rxn_names, fraction_of_optimum=0.9):
    """
    Runs Flux Variability Analysis (FVA) on the model given a Reaction List and a Fraction of Optimum (FO) Value.    
    """
    try:
        # model = model_manager.get_current_model()
        rxn_obj_list = []

        # if not model_manager.objective:
        #     return {"error": "No Objective Function is set for the model."}

        for name in rxn_names:
            match = next(
                (rxn for rxn in model.reactions if name.lower() in rxn.name.lower()),
                None
            )
            if match is None:
                raise ValueError(f"Reaction name '{name}' not found in model.")
            rxn_obj_list.append(match)

        fva_result = flux_variability_analysis(model, rxn_obj_list, fraction_of_optimum=fraction_of_optimum)

        fva_df = fva_result.reset_index()
        fva_df.insert(0, "Reaction Name", [rxn.name for rxn in rxn_obj_list])
        fva_df.columns = ["Reaction Name", "Reaction ID", "Maximum Flux", "Minimum Flux"]

        if len(fva_df) > 5:
            output_dir = "outputs"
            os.makedirs(output_dir, exist_ok=True)
            csv_path = os.path.join(output_dir, f"fva_result_xyz.csv")
            fva_df.to_csv(csv_path, index=False)

            return {
                "fraction_of_optimum": fraction_of_optimum,
                "fva_csv_file": f"file://{csv_path}",
                "fva_output": fva_df.iloc[:5,:].to_dict(orient="records"),
                "message": f"FVA result has {len(fva_df)} entries, saved to CSV."
            }
        else:
            return {
                "fraction_of_optimum": fraction_of_optimum,
                "fva_output": fva_df.to_dict(orient="records")
            }

    except Exception as e:
        return {"error": str(e)}
    
run_fva(['Phosphofructokinase', 'Pyruvate formate lyase', 'Glucose-6-phosphate isomerase', 'Phosphoglycerate kinase', '6-phosphogluconolactonase', 'Acetaldehyde dehydrogenase'], 0.5)

{'fraction_of_optimum': 0.5,
 'fva_csv_file': 'file://outputs\\fva_result_xyz.csv',
 'fva_output': [{'Reaction Name': 'Phosphofructokinase',
   'Reaction ID': 'PFK',
   'Maximum Flux': 0.0,
   'Minimum Flux': 93.90368533529579},
  {'Reaction Name': 'Pyruvate formate lyase',
   'Reaction ID': 'PFL',
   'Maximum Flux': 0.0,
   'Minimum Flux': 28.826782445182594},
  {'Reaction Name': 'Glucose-6-phosphate isomerase',
   'Reaction ID': 'PGI',
   'Maximum Flux': -30.1661329189748,
   'Minimum Flux': 9.910423045535737},
  {'Reaction Name': 'Phosphoglycerate kinase',
   'Reaction ID': 'PGK',
   'Maximum Flux': -18.838427229012915,
   'Minimum Flux': -5.479575240842752},
  {'Reaction Name': '6-phosphogluconolactonase',
   'Reaction ID': 'PGL',
   'Maximum Flux': 0.0,
   'Minimum Flux': 40.076555964510405}],
 'message': 'FVA result has 6 entries, saved to CSV.'}

In [9]:
output_dir = os.path.join(os.getcwd(), 'outputs')
# os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, f"fva_result.csv")
# fva_df.to_csv(csv_path, index=False)
print(csv_path)

e:\INTERNSHIP\IITM\metabolic\outputs\fva_result.csv


#### TOOL TESTING

In [16]:
def reaction_info(rxn_name: str) -> dict:
    """
    Returns information about a specific reaction in the model.
    This function simulates fetching reaction data from a database or API.
    """
    try:
        # model = model_manager.get_current_model()
        for rxn in model.reactions:
            if rxn.name == rxn_name:
                rxn_name = rxn.id
                break
        reaction = model.reactions.get_by_id(rxn_name)    
        return {
            "Reaction id": reaction.id,
            "name": reaction.name,
            "Stochiometry": reaction.build_reaction_string(),
            "GPR" : str(reaction.gpr) or "Not Set",
            "lower_bound": reaction.lower_bound,
            "upper_bound": reaction.upper_bound,
        }
    except Exception as e:
        return {"error": str(e)}

In [17]:
str([v for k,v in model.compartments.items()])

"['extracellular space', 'cytosol']"

In [18]:
reaction_info('Phosphofructokinase')

{'Reaction id': 'PFK',
 'name': 'Phosphofructokinase',
 'Stochiometry': 'atp_c + f6p_c --> adp_c + fdp_c + h_c',
 'GPR': 'b3916 or b1723',
 'lower_bound': 0.0,
 'upper_bound': 1000.0}

In [19]:
model.genes[0]

Gene identifier,b1241
Name,adhE
Memory address,0x20bedd55180
Functional,True
In 2 reaction(s),"ALCD2x, ACALD"


In [20]:
model.reactions.EX_glc__D_e

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x20bede2bb20
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


In [21]:
metabolite = model.metabolites.get_by_id('glc__D_e')
', '.join([rxn.id for rxn in metabolite.reactions])

'GLCpts, EX_glc__D_e'

In [22]:
reaction = model.reactions.get_by_id(model.reactions[1].id)
print(str(reaction.gpr))
reaction

((b0902 and b0903) and b2579) or (b0902 and b0903) or (b0902 and b3114) or (b3951 and b3952)


Reaction identifier,PFL
Name,Pyruvate formate lyase
Memory address,0x20bedd56ad0
Stoichiometry,coa_c + pyr_c --> accoa_c + for_c Coenzyme A + Pyruvate --> Acetyl-CoA + Formate
GPR,((b0902 and b0903) and b2579) or (b0902 and b0903) or (b0902 and b3114) or (b3951 and b3952)
Lower bound,0.0
Upper bound,1000.0


In [23]:
pgi_gene = model.genes.get_by_id("b1241")
', '.join([rxn.id for rxn in pgi_gene.reactions])

'ALCD2x, ACALD'

In [24]:
[i.split(",") for i in """rxn_id,lower_bound,upper_bound
EX_glc__D_e,-10.0,0.0
EX_o2_e,-20.0,0.0
""".split('\n')]

[['rxn_id', 'lower_bound', 'upper_bound'],
 ['EX_glc__D_e', '-10.0', '0.0'],
 ['EX_o2_e', '-20.0', '0.0'],
 ['']]

In [25]:
bounds_dict = {
    "EX_glc__D_e": (-10.0, 0.0),
    "EX_o2_e": (-20.0, 0.0)
}

In [26]:
# def bounds_data_for_FBA(csv_file_path: str) -> dict:
#     import pandas as pd
#     try:
#         df = pd.read_csv(csv_file_path)
#         bounds_data = df.to_dict(orient='records')
#         return bounds_data
#     except Exception as e:
#         return {"error": str(e)}

def run_fba(bounds: dict = {}) -> str:
    for rxn_id, bounds in bounds.items():
        if rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).bounds = (bounds[0], bounds[1])

    solution = model.optimize()
    print(solution.status, "Hello", solution.fluxes[:5], solution.shadow_prices[:5])
    # return f"Objective value: {solution.objective_value}"

run_fba(bounds_dict)

optimal Hello PFK    3.356
PFL    0.000
PGI    3.356
PGK   -6.712
PGL    0.000
Name: fluxes, dtype: float64 glc__D_e    0.0
gln__L_c    0.0
gln__L_e    0.0
glu__L_c    0.0
glu__L_e    0.0
Name: shadow_prices, dtype: float64


In [27]:
from models import ModelManager
model_manager = ModelManager()
model_manager.load_sbml(r"E:\INTERNSHIP\IITM\metabolic\uploads\e_coli_core.xml")

'E:\\INTERNSHIP\\IITM\\metabolic\\uploads\\e_coli_core'

In [28]:
def model_data() -> dict:
    """
    Returns metadata for a given a model.
    This function simulates fetching metadata from a database or API.
    """
    try:
        model = model_manager.get_current_model()

        for rxn in model.reactions:
            if rxn.lower_bound is None:
                rxn.lower_bound = -1000.0
            if rxn.upper_bound is None:
                rxn.upper_bound = 1000.0

        data = {
            "model_id": str(model.id),
            # "model_name": str(model.name) if model.name else "unknown",
            "objective_reaction": str(model.objective.expression) if model.objective.expression else "Not Set Yet",
            "reactions_count": len(model.reactions),
            "metabolites_count": len(model.metabolites),
            "genes_count": len(model.genes),
            "groups_count": len(model.groups),
            "compartments_count": len(model.compartments),
            "Compartments": str([v.name for k,v in model.compartments.items()]),
        }
        return data
    except Exception as e:
        return {
            "error": str(e),
            "model_id": model_manager.current_model_id
        }


In [29]:
import pandas as pd
csv = pd.read_csv(r"C:\Users\Aadhitya Sriram\Downloads\e_coli_bounds.csv")
print(csv)
print(csv.values.tolist())

      reaction   lval   uval
0  EX_glc__D_e  -10.0    0.0
1      EX_o2_e  -20.0    0.0
[['EX_glc__D_e', -10.0, 0.0], ['EX_o2_e', -20.0, 0.0]]


#### LLM INTEGRATION

In [3]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
class HuggingFaceLLM:
    def __init__(self, model_name="mistralai/Mistral-7B-Instruct-v0.1"):
        self.model_name = model_name
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto"
        )
        self.generator = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device == "cuda" else -1
        )

    def chat(self, prompt: str, max_new_tokens: int = 512) -> str:
        output = self.generator(
            prompt,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=self.tokenizer.eos_token_id
        )
        return output[0]["generated_text"]

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
llm = HuggingFaceLLM(model_name="Qwen/Qwen3-1.7B")

e:\Conda\envs\mecha\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

#### Simulating Deletions

In [18]:
import pandas
from time import time

from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

from models import ModelManager
model_manager = ModelManager()
model_manager.load_model_by_id('e_coli_core')
model = model_manager.get_current_model()

In [ ]:
reaction_names = ['Phosphofructokinase', 'Pyruvate formate lyase', 'Glucose-6-phosphate isomerase']

valid_rxns = []
seen = set()  # avoid duplicates
for rxn in model.reactions:
    if rxn.name in reaction_names and rxn.id not in seen:
        valid_rxns.append(rxn)
        seen.add(rxn.id)

valid_rxns

[<Reaction PFK at 0x1e5d5cea8f0>,
 <Reaction PFL at 0x1e5d5cea050>,
 <Reaction PGI at 0x1e5d5ce9840>]

In [10]:
[rxn.id for rxn in model.reactions[:10]]

['PFK',
 'PFL',
 'PGI',
 'PGK',
 'PGL',
 'ACALD',
 'AKGt2r',
 'PGM',
 'PIt2r',
 'ALCD2x']

In [4]:
cobra_model = load_model("textbook")

print('complete model: ', cobra_model.optimize())
with cobra_model:
    # reaction = cobra_model.reactions.PFK
    for rxn in cobra_model.reactions:
        if rxn.id == "PFK":
            reaction = rxn
            break
    reaction.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())


complete model:  <Solution 0.874 at 0x24c11badd80>
pfk knocked out:  <Solution 0.704 at 0x24c11c239d0>


Give as a list

In [7]:
from cobra.flux_analysis import single_gene_deletion, double_gene_deletion
from cobra.flux_analysis import single_reaction_deletion, double_reaction_deletion

def gene_knockout_simulation(gene_names: list[str], type: str = "single") -> dict:
    """
    Performs single or double gene knockout simulations on the loaded metabolic model.
    """
    try:
        model = model_manager.get_current_model()
        valid_genes = [g for g in gene_names if g in model.genes]
        if not valid_genes:
            return {"error": "None of the provided genes are valid in this model."}

        if type == "single":
            result = single_gene_deletion(model, gene_list=valid_genes)
        elif type == "double":
            result = double_gene_deletion(model, gene_list=valid_genes)
        else:
            return {"error": "Invalid type. Choose 'single' or 'double'."}

        result = result.rename(columns={
            "growth": "Post-KO Growth",
            "ids": "Gene(s)",
            "status": "Solver Status"
        })

        if len(result) > 5:
            file_path = "/tmp/gene_knockout_result.csv"
            result.to_csv(file_path, index=False)
            return {"file": file_path, "note": "Too many results to display. Download CSV."}

        return result.iloc[:5].to_dict(orient="records")

    except Exception as e:
        return {"error": str(e)}

In [ ]:
def reaction_knockout_simulation(reaction_ids: list[str], type: str = "single") -> dict:
    """
    Performs single or double reaction knockout simulations on the loaded metabolic model.
    """
    try:
        model = model_manager.get_current_model()
        valid_rxns = [r for r in reaction_ids if r in model.reactions]
        if not valid_rxns:
            return {"error": "None of the provided reactions are valid in this model."}

        if type == "single":
            result = single_reaction_deletion(model, reaction_list=valid_rxns)
        elif type == "double":
            result = double_reaction_deletion(model, reaction_list=valid_rxns)
        else:
            return {"error": "Invalid type. Choose 'single' or 'double'."}

        result = result.rename(columns={
            "growth": "Post-KO Growth",
            "ids": "Reaction(s)",
            "status": "Solver Status"
        })

        if len(result) > 5:
            file_path = "/tmp/reaction_knockout_result.csv"
            result.to_csv(file_path, index=False)
            return {"file": file_path, "note": "Too many results to display. Download CSV."}

        return result.iloc[:5].to_dict(orient="records")

    except Exception as e:
        return {"error": str(e)}

In [9]:
reaction_knockout_simulation(['PFK', 'PGI'], type="single")

{'error': 'None of the provided reactions are valid in this model.'}

#### Flux Sampling

In [1]:
from cobra.io import load_model
from cobra.sampling import sample

model = load_model("textbook")
s = sample(model, 100)
s.head()

,ACALD,ACALDt,ACKr,ACONTa,ACONTb,ACt2r,ADK1,AKGDH,AKGt2r,ALCD2x,...,RPI,SUCCt2_2,SUCCt3,SUCDi,SUCOAS,TALA,THD2,TKT1,TKT2,TPI
0,-3.890821,-0.792777,-2.851561,8.749809,8.749809,-2.851561,7.283112,1.392740,-0.106940,-3.098044,...,-0.430010,5.499806,7.698045,549.353168,-1.392740,0.419973,8.987019,0.419973,0.415937,9.491228
1,-0.508250,-0.417557,-0.579870,14.142261,14.142261,-0.579870,12.410068,3.211441,-1.190809,-0.090693,...,-0.431297,12.904390,14.103749,804.607737,-3.211441,0.386210,5.013765,0.386210,0.368078,9.534583
2,-0.317894,-0.009309,-1.692664,8.385905,8.385905,-1.692664,2.588217,0.856310,-1.545613,-0.308585,...,-1.556312,19.144189,19.843284,815.325093,-0.856310,1.526594,0.321619,1.526594,1.514643,7.766499
3,-1.804146,-1.123270,-0.377417,7.358081,7.358081,-0.377417,1.249721,2.050678,-0.291257,-0.680876,...,-3.336967,38.220412,39.767240,844.068378,-2.050678,3.331725,0.025116,3.331725,3.329617,6.185412
4,-2.840410,-1.745393,-0.013737,9.007029,9.007029,-0.013737,14.702444,3.380720,-0.744910,-1.095018,...,-3.460033,21.305473,21.823486,707.864824,-3.380720,3.458024,17.627957,3.458024,3.457216,6.376081


In [ ]:
print("One process:")
%time s = sample(model, 1000)
print("Two processes:")
%time s = sample(model, 1000, processes=2)
print("ACHR")
%time s = sample(model, 100, method="achr")

One process:
CPU times: total: 188 ms
Wall time: 3.5 s
Two processes:
CPU times: total: 0 ns
Wall time: 4.1 s
ACHRT
CPU times: total: 93.8 ms
Wall time: 490 ms


In [ ]:
from cobra.sampling import OptGPSampler, ACHRSampler
achr = ACHRSampler(model, thinning=10)
optgp = OptGPSampler(model, processes=4)

In [ ]:
s1 = achr.sample(100)
s2 = optgp.sample(100)
print("s1:", s1)
print("s2:", s2)

s1:        ACALD    ACALDt      ACKr     ACONTa     ACONTb     ACt2r       ADK1  \
0  -4.141322 -2.515126 -0.360114   8.360897   8.360897 -0.360114  11.275434   
1  -2.794019 -2.127295 -1.932392   7.665680   7.665680 -1.932392  10.802762   
2  -3.261962 -2.165435 -0.745168   7.851490   7.851490 -0.745168   6.937721   
3  -4.309282 -2.204211 -0.490739   8.308006   8.308006 -0.490739  11.327321   
4  -4.411390 -2.208213 -1.619287   7.991046   7.991046 -1.619287  12.886279   
..       ...       ...       ...        ...        ...       ...        ...   
95 -1.153290 -0.910909 -1.191121   8.870089   8.870089 -1.191121   3.611420   
96 -1.071335 -0.906100 -1.107278  10.566670  10.566670 -1.107278   3.628939   
97 -1.266864 -1.014282 -0.873268  12.163036  12.163036 -0.873268   2.964187   
98 -1.328069 -0.946664 -3.994312   9.824124   9.824124 -3.994312   1.432650   
99 -1.877173 -1.016000 -1.260958  10.349873  10.349873 -1.260958   1.839877   

       AKGDH    AKGt2r    ALCD2x  ...       RPI

In [9]:
import numpy as np

bad = np.random.uniform(-1000, 1000, size=len(model.reactions))
achr.validate(np.atleast_2d(bad))

array(['le'], dtype='<U3')

Engine recommendation

In [5]:
def recommend_sampling_config(model):
    import multiprocessing
    import psutil

    n_rxns = len(model.reactions)
    cpu_cores = multiprocessing.cpu_count()
    ram_gb = psutil.virtual_memory().total / 1e9

    # Choose method
    if n_rxns < 500:
        method = "achr"
    elif cpu_cores >= 4:
        method = "optgp"
    else:
        method = "achr"

    # Thinning logic
    thinning = max(10, int(n_rxns / 5))
    thinning = min(thinning, 500)  # cap for efficiency

    # Processes (only for optgp)
    processes = None
    if method == "optgp":
        if cpu_cores >= 8:
            processes = cpu_cores // 2
        elif cpu_cores >= 4:
            processes = 2
        else:
            processes = 1
        if ram_gb < 4:
            processes = min(processes, 2)

    return {
        "method": method,
        "thinning": thinning,
        "processes": processes
    }

Tools

In [ ]:
import os
def sample_metabolic_model(model, n_samples=1000):
    """
    Samples a metabolic model given the number of samples.
    """
    config = recommend_sampling_config(model)
    if config["method"] == "achr":
        sampler = ACHRSampler(model, thinning=config["thinning"])
    else:
        sampler = OptGPSampler(model, thinning=config["thinning"], processes=config["processes"])
    output_dir = os.path.join(os.getcwd(), 'outputs/flux_sampling', 'flux_sampling_result.csv')
    samples = sampler.sample(n_samples)
    samples.to_csv(output_dir, index=False)
    subset = samples.iloc[:5, :5]
    return {
        "status": "success",
        "n_samples": n_samples,
        "method": config["method"],
        "thinning": config["thinning"],
        "processes": config["processes"],
        "save_path": output_dir,
        "samples": subset.to_dict(orient="records")
    }

In [19]:
sample_metabolic_model(model, n_samples=100)

{'status': 'success',
 'n_samples': 100,
 'method': 'achr',
 'thinning': 19,
 'processes': None,
 'save_path': 'e:\\INTERNSHIP\\IITM\\Agenti_AI_for_COBRA\\outputs/flux_sampling\\flux_sampling_result.csv',
 'samples': [{'PFK': 11.706358484798232,
   'PFL': 3.9450800224282765,
   'PGI': 9.262693797787877,
   'PGK': -19.55242666308508,
   'PGL': 0.6166879028443322},
  {'PFK': 11.434540491023906,
   'PFL': 3.923164904782254,
   'PGI': 8.623991459802802,
   'PGK': -19.261735471079817,
   'PGL': 1.2273377896202884},
  {'PFK': 11.294957660820362,
   'PFL': 3.868403557895533,
   'PGI': 8.70418912244538,
   'PGK': -19.51544120873183,
   'PGL': 1.282597316368462},
  {'PFK': 11.492479490620893,
   'PFL': 3.989337813477387,
   'PGI': 8.7054315595132,
   'PGK': -19.518109890891107,
   'PGL': 1.2809023424307147},
  {'PFK': 10.246731218489147,
   'PFL': 3.7011774516911067,
   'PGI': 8.771747739099228,
   'PGK': -19.380863553209778,
   'PGL': 1.1355677583972799}]}

In [11]:
print(recommend_sampling_config(model))

{'method': 'achr', 'thinning': 19, 'processes': None}


In [21]:
import pandas as pd
file_path = r"E:\INTERNSHIP\IITM\Agenti_AI_for_COBRA\outputs\flux_sampling\flux_sampling_result.csv"
pd.read_csv(file_path).shape

(100, 95)

#### HE

In [22]:
for i in model.genes:
    print(i.name)

adhE
mhpF
s0001
purT
tdcD
ackA
acnA
acnB
adk
lpd
sucB
sucA
kgtP
frmA
adhP
atpA
atpG
atpF
atpE
atpI
atpB
atpH
atpC
atpD
gltA
cydA
cydB
cbdB
cbdA
lldP
glcA
eno
fbaA
ydjI
fbaB
glpX
fbp
focB
focA
frdC
frdA
frdB
frdD
manZ
manX
ptsI
ptsH
manY
fumC
fumB
fumA
dctA
zwf
gapA
ptsG
crr
malX
puuA
glnA
glnQ
glnH
glnP
gdhA
glsB
glsA
pabB
gltD
gltB
gltP
gnd
aqpZ
icd
aceA
ldhA
dld
aceB
glcB
mdh
maeA
maeB
nuoI
nuoM
nuoJ
nuoC
nuoB
nuoF
nuoN
nuoH
nuoK
nuoG
nuoE
nuoA
nuoL
pntA
sthA
pntB
amtB
aceE
aceF
pfkA
pfkB
tdcE
grcA
pflD
pflA
pflC
pflB
pgi
pgk
pgl
gpmM
ytjC
gpmA
pitA
pitB
ppc
pck
ppsA
pta
eutD
pykF
pykA
rpe
sgcE
rpiA
rpiB
sdhC
sdhD
sdhB
sdhA
sucD
sucC
talB
talA
tktB
tktA
tpiA
